# Initialization

In [2]:
import os
import grasp
import numpy as np
from astropy import units as u
from matplotlib import pyplot as plt
dr3 = grasp.dr3()
gc = grasp.Cluster('ngc6121')

try:
    device_name = os.getenv('COMPUTERNAME')
    if device_name == 'DESKTOP-Work':
        tn1 = '20250402_204446'
        tn2 = '20250402_204448'
        acs = grasp.load_data(tn1)
        pcs = grasp.load_data(tn2)
    elif device_name == 'LAPTOP-Work':
        tn1 = '20250401_164228'
        tn2 = '20250401_164231'
        tn3 = '20250414_095328'
        pcs = grasp.load_data(tn1)
        acs = grasp.load_data(tn2)
        fas = grasp.load_data(tn3)
    else:
        raise EnvironmentError("Unknown device name")
except Exception:
    astrometry_query = "SELECT source_id, ra, ra_error, dec, dec_error, parallax, parallax_error, pmra, pmra_error, pmdec, pmdec_error, \
                        radial_velocity, radial_velocity_error, bp_rp, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, teff_gspphot, ra_dec_corr, pmra_pmdec_corr \
                        FROM gaiadr3.gaia_source \
                        WHERE CONTAINS(POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),CIRCLE('ICRS',245.897,-26.526,0.86))=1 \
                        AND parallax IS NOT NULL AND parallax>0.531632110579479 AND parallax<0.5491488193300\
                        AND abs(parallax_error/parallax)<0.50\
                        AND abs(pmra_error/pmra)<0.30 \
                        AND abs(pmdec_error/pmdec)<0.30 \
                        AND pmra IS NOT NULL AND abs(pmra)>0 \
                        AND pmdec IS NOT NULL AND abs(pmdec)>0 \
                        AND pmra BETWEEN -13.742720 AND -11.295338 \
                        AND pmdec BETWEEN -20.214805 AND -17.807517"
    
    photometry_query = "SELECT source_id, ra, ra_error, dec_error, dec, parallax, parallax_error, pmra, pmra_error, pmdec, pmdec_error, radial_velocity, radial_velocity_error, \
                        bp_rp, phot_g_mean_mag, phot_bp_rp_excess_factor, teff_gspphot, ra_dec_corr, pmra_pmdec_corr \
                        FROM gaiadr3.gaia_source \
                        WHERE CONTAINS(POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),CIRCLE('ICRS',245.8958,-26.5256,0.86))=1 \
                        AND parallax IS NOT NULL AND parallax>0.531632110579479 AND parallax<0.5491488193300\
                        AND ruwe < 1.15 \
                        AND phot_g_mean_mag > 11 \
                        AND astrometric_excess_noise_sig < 2 \
                        AND pmra BETWEEN -13.742720 AND -11.295338 \
                        AND pmdec BETWEEN -20.2148 AND -17.807517"
    acs = dr3.free_query(astrometry_query, save=True)
    acs = grasp.Sample(acs, gc)
    pcs = dr3.free_query(photometry_query, save=True)
    pcs = grasp.Sample(pcs, gc)
    fas = dr3.get_astrometry(0.86, 'ngc6121', save=True)
    print("\nWARNING! Remember to updates tn after running the new query!!!")

PermissionError: [Errno 13] Permission denied: '/media/pietrof/s820/graspdata'

In [ ]:
aps = acs.join(pcs)
aps.gc.dist = 1851 * u.pc # Baumgardt, Vasiliev: 2021 # pc
f = grasp.load_base_formulary()
aps.info()

# Data visualization

In [ ]:
grasp.plots.colorMagnitude(aps)

In [ ]:
grasp.plots.histogram(aps.parallax)

In [ ]:
grasp.plots.doubleHistScatter(aps.ra, aps.dec, xlabel='RA', ylabel='Dec')

# Angular Separation Analysis

The `Great Circle` formula versus the `Vincenty Formula` for the computation of distances on a sphere

## $\theta_V$   vs   $\theta_{GC}$

In [ ]:
f.substitute('Angular separation', {'alpha_{0}': aps.gc.ra.to(u.rad).value, 'delta_{0}':aps.gc.dec.to(u.rad).value})

from sympy import atan2
atan_arg_1 = 'sqrt((cos(delta_1) * sin((alpha_0 - alpha_1)/2))**2 + (cos(delta_0)*sin(delta_1) - sin(delta_0)*cos(delta_1)*cos((alpha_0 - alpha_1)/2 ))**2)'
atan_arg_2 = '(sin(delta_0)*sin(delta_1) + cos(delta_0)*cos(delta_1)*cos((alpha_0 - alpha_1)/2))'
atan = atan2(atan_arg_1, atan_arg_2)
f.add_formula('Vincenty angsep', atan)
f.substitute('Vincenty angsep', {'alpha_0': aps.gc.ra.to(u.rad).value, 'delta_0':aps.gc.dec.to(u.rad).value})


f.angular_separation

In [ ]:
f['Vincenty angsep']

In [ ]:
f.var_order('Angular Separation')
print('')
f.var_order('Vincenty angsep')

In [ ]:
ra, dec = ((aps.ra * u.deg).to(u.rad).value, (aps.dec * u.deg).to(u.rad).value)
print("Great Circle Distance computation\n")
theta_1 = f.compute('Angular Separation', data={'alpha_{1}': ra, 'delta_{1}': dec}, asarray=True)
print("\nVincenty Distance computation\n")
theta_2 = f.compute('Vincenty angsep', data={'alpha_1': ra, 'delta_1': dec}, asarray=True)

grasp.plots.doubleHistScatter(theta_2, theta_1, xlabel='Vincenty Formula', ylabel='Angular Separation')

In [ ]:
t_ratio = theta_2/theta_1
err_t = (1-t_ratio) * 100
out = grasp.plots.histogram(err_t, kde=True, kde_kind='lorentzian', xlabel=r'$\theta_V\,/\,\theta_G$', out=True)
fit = out['kde']
print(f"A = {fit[0]:.2f}  ;  lambda = {fit[1]:.2f}")

### ADQL computed Angular Separation : $\theta_{G}$

In [ ]:
query1: str = "SELECT source_id, DISTANCE(245.897,-26.526, ra, dec) AS ang_sep \
    FROM gaiadr3.gaia_source \
    WHERE CONTAINS(POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),CIRCLE('ICRS',245.897,-26.526,0.86))=1 \
    AND parallax IS NOT NULL AND parallax>0.531632110579479 AND parallax<0.5491488193300\
    AND abs(parallax_error/parallax)<0.50\
    AND abs(pmra_error/pmra)<0.30 \
    AND abs(pmdec_error/pmdec)<0.30 \
    AND pmra IS NOT NULL AND abs(pmra)>0 \
    AND pmdec IS NOT NULL AND abs(pmdec)>0 \
    AND pmra BETWEEN -13.742720 AND -11.295338 \
    AND pmdec BETWEEN -20.214805 AND -17.807517"

query2: str = "SELECT source_id, DISTANCE(245.897,-26.526, ra, dec) AS ang_sep \
    FROM gaiadr3.gaia_source \
    WHERE CONTAINS(POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),CIRCLE('ICRS',245.897,-26.526,0.86))=1 \
    AND parallax IS NOT NULL AND parallax>0.531632110579479 AND parallax<0.5491488193300\
    AND abs(parallax_error/parallax)<0.50\
    AND abs(pmra_error/pmra)<0.30 \
    AND abs(pmdec_error/pmdec)<0.30 \
    AND pmra IS NOT NULL AND abs(pmra)>0 \
    AND pmdec IS NOT NULL AND abs(pmdec)>0 \
    AND pmra BETWEEN -13.742720 AND -11.295338 \
    AND pmdec BETWEEN -20.214805 AND -17.807517"

aps2 = dr3.free_query(query1)
pcs2 = dr3.free_query(query2)

aps2 = aps2.join(pcs2)

## $r_{2D}(\theta_V)$ vs $r_{2D}(\theta_{GC})$

In [ ]:
f.substitute('radial_distance_2d', {'r_{c}': aps.gc.dist.value})
f.radial_distance_2d

In [ ]:
f.var_order('radial_distance_2d')

In [ ]:
print(r'Computation using $\theta_{GC}$')
r2d_1 = f.compute('radial_distance_2d', data={'theta': theta_1}, asarray=True) # Great Circle
print('')
print(r'Computation using $\theta_{V}$')
r2d_2 = f.compute('radial_distance_2d', data={'theta': theta_2}, asarray=True) # Vincenty Formula

r_ratio = r2d_2/r2d_1

In [ ]:
grasp.plots.doubleHistScatter(r2d_2, r2d_1, xlabel=r'$r_{2d}(\theta_{V})$', ylabel=r'$r_{2d}(\theta_{GC})$')

In [ ]:
grasp.plots.histogram(1-r_ratio[r_ratio<400], kde=True, kde_kind='lorentzian', xlabel=r'$r_{2d}(\theta_{V})\,/\,r_{2d}(\theta_{GC})$')

In [ ]:
reg_e = grasp.stats.fit_distribution(r_ratio[r_ratio<400], 'exponential', False)
reg_p = grasp.stats.fit_distribution(r_ratio[r_ratio<400], 'power', False)

# Radial Distance $R$

### Computing $d$ and $r_x$

In [ ]:
f.los_distance

In [ ]:
f.gc_z_coordinate


In [ ]:
f.radial_distance_3d

In [ ]:
rt_pc = np.tan(aps.gc.rt.to(u.rad).value) * aps.gc.dist.to(u.pc)
print(f"Tidal radius of {aps.gc.id} = {rt_pc:.2f}")

In [ ]:
f.var_order('los_distance')
f.var_order('gc_z_coordinate')
f.substitute('gc_z_coordinate', {'r_{c}': aps.gc.dist.value})
print('')
r_x = f.compute('los_distance', data={'omega': aps.parallax}, asarray=True) * 1000
print('')
d = f.compute('gc_z_coordinate',data={'r_{x}': r_x}, asarray=True)

### $R(\theta_V)$

In [ ]:
f.var_order('radial_distance_3d')

In [ ]:
R_V = f.compute('radial_distance_3d', data={'d': d,'r_{2}': r2d_2,}, asarray=True)
grasp.plots.histogram(R_V, xlabel=r'$R(\theta_V)$')

### $R(\theta_{GC}$)

In [ ]:
R_GC = f.compute('radial_distance_3d', {'d': d,'r_{2}': r2d_1,}, asarray=True)
rgc_hist = grasp.plots.histogram(R_GC, xlabel=r'$R(\theta_{GC})$', kde=True, kde_kind='gaussian', out=True, kde_verbose=False)

### $R(\theta_V)$ vs $R(\theta_{GC})$

In [ ]:
grasp.plots.spatial(aps, color=R_V/R_GC, colorbar=True, colorbar_label=r'$r_{2d}(\theta_{GC})$', cmap='plasma_r', title='Radial distance using GC formula')

In [ ]:
print(f"ratio mean = {np.mean(R_V/R_GC):.2f} ; ratio std = {np.std(R_V/R_GC):.2f}")

**Conclusions**

The vincenty formula computes the angular separation between two points on an ellipsoid, so, in the case of computing<br>
angular separations between stars in the celestial sphere, which geometry is, effectively, a unit sphere, the Great Circle<br>
formula is the correct one to use, as it computes the distances on a spheric geometry<br>

*Overall*, defined the ratio 

> $\bar{r} = \dfrac{R(\theta_{V})}{R(\theta_{GC})}$

its mean and standard deviation

> $\big<\bar{r}\big> = 0.96$

> $\sigma(\bar{r}) = 0.08$

indicates that, for the problem at hand, the two equations provide almost equal results, provided some outlier data.

In the end, using the GC formula is recommended, as it is computed on the actual geometry we are considering

### Ulterior analysis with cartesian coordinate transformations

In [ ]:
from grasp.functions import CartesianConversion
import sympy as sp
ra0 ,dec0 = sp.symbols('ra_0 dec_0')

#cv = CartesianConversion(aps.gc.ra.value, aps.gc.dec.value)

# Dynamics 

### PMRA ($\mu_\alpha$) mean

In [ ]:
pmra = aps.pmra
pmra_kde = grasp.plots.histogram(pmra, xlabel='pmra', kde=True, kde_kind='gaussian', out=True)['kde']
pmra_mean = pmra_kde[1]
print(f"{pmra_mean = :.4f}")

### PMDEC ($\mu_{\delta^*}$) mean

In [ ]:
pmdec = aps.pmdec
pmdec_kde = grasp.plots.histogram(pmdec, xlabel='pmra', kde=True, kde_kind='gaussian', out=True)['kde']
pmdec_mean = pmdec_kde[1]
print(f"{pmdec_mean = :.4f}")

## Gaussian Mixture Model attempt

# Prove

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel('RA (deg)')
ax.set_ylabel('Dec (deg)')
ax.set_zlabel('Distance (pc)')

# Use X, Y, and d for 3D coordinates
X = aps.ra - gc.ra.value
Y = aps.dec - gc.dec.value
Z = d  # Distance (pc)
accel_x = aps.pmra - pmra_mean
accel_y = aps.pmdec - pmdec_mean
pm = np.sqrt(accel_x**2 + accel_y**2)

Axes3D.scatter3D = ax.scatter(X, Y, Z, c=pm, cmap='plasma', marker='o', alpha=0.5)
# Plot the quiver in 3D
arrows = ax.quiver(X, Y, Z, accel_x, accel_y, np.zeros_like(accel_x), alpha=0.5)

# Add a color bar
cbar = fig.colorbar(arrows, ax=ax, shrink=0.5, aspect=10)
cbar.set_label('Proper Motion Magnitude (mas/yr)')